<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>Editado por Equipo Docente IIC2233 2018-1, 2018-2, y 2019-2. Basado en documento de Nebil Kawas 2017-2.</font><br>
<font size='1'>Incluye partes de un material confeccionado por Karim Pichara y Christian Pieringer en 2015.</font>
<br>
</p>

## Decoradores de funciones

En diseño de *software*, un ***decorador*** es un patrón de diseño en el que se añade una funcionalidad a un objeto, **sin tener que reescribir el código original**. Los decoradores de funciones aplican esa idea: permiten tomar una función ya implementada, agregar algún comportamiento o datos adicionales, y retornar una nueva función. 

Podemos ver los decoradores como funciones que reciben una función `f1` cualquiera, y retornan una función `f2` distinta. Por ejemplo, si nuestro decorador se llama `decorator`, para obtener la función modificada que queremos y asignarla a la misma función actual, simplemente escribimos `f1 = decorator(f1)`. Con esto, nuestra función `f1` ahora queda con los nuevos datos y comportamientos agregados. 

Un beneficio de los decoradores es que evitan la necesidad de modificar el código de la función original, por lo que si necesitamos volver a la versión inicial de la función simplemente quitamos el llamado al decorador.

Empezaremos por algo sencillo: definamos un decorador *identidad*. Le decimos *identidad* porque, simplemente, devuelve una función (el *wrapper*) que, una vez llamado, ejecutará la función original. La función `deco_funcion` implementa este decorador.

In [14]:
def deco_function(funcion_original):
    print("[deco_function] Entrando... ")

    # Esta función 'wrapper' funciona como un "envoltorio", que en este caso solamente llama a la función original
    def wrapper_function():
        print("[wrapper_function] Entrando... ")
        funcion_original()
        print("[wrapper_function] Saliendo... ")

    print("[deco_function] Saliendo... ")
    return wrapper_function

Supongamos que estamos en nuestra época favorita del año, como pueden ser las Fiestas Patrias. Ahora, definimos una función dieciochera.

In [15]:
def print_paya():
    print("¡Aro, aro, aro!")

Llamamos a esta función recién definida.

In [16]:
print_paya()

¡Aro, aro, aro!


Le entregamos la función al decorador `deco_function` —recordemos que el decorador también es una función— para crear una **nueva** función.

In [17]:
nuevo_print_paya = deco_function(print_paya)

[deco_function] Entrando... 
[deco_function] Saliendo... 


Este llamado **define** una `wrapper_function` (no la invoca). Retorna esta `wrapper_function`, la cual llama a `funcion_original`, y que ahora será `nuevo_print_paya`.

La función decorada, `nuevo_print_paya` tiene el nuevo comportamiento.

In [18]:
nuevo_print_paya()

[wrapper_function] Entrando... 
¡Aro, aro, aro!
[wrapper_function] Saliendo... 


La función original sigue teniendo el comportamiento original.

In [19]:
print_paya()

¡Aro, aro, aro!


También podemos **reemplazar la función original** por la función decorada.

In [20]:
print_paya = deco_function(print_paya)

[deco_function] Entrando... 
[deco_function] Saliendo... 


Y obtendremos el mismo resultado.

In [21]:
print_paya()

[wrapper_function] Entrando... 
¡Aro, aro, aro!
[wrapper_function] Saliendo... 


### Decoradores con *azúcar sintáctico*  

Una forma equivalente, pero más rápida y legible de decorar funciones es escribiendo el **nombre del decorador arriba del encabezado de la función** anteponiendo un `@`. Es la misma sintaxis que usamos cuando queremos crear *properties*, de hecho, `property` es un decorador que ya hemos usado.

Si partimos con la función original:

In [29]:
def print_paya():
    print("¡Aro, aro, aro!")
    
print_paya()

¡Aro, aro, aro!


Nuestra paya decorada de esta manera, quedaría:

In [25]:
@deco_function
def print_paya():
    print("¡Aro, aro, aro!")

[deco_function] Entrando... 
[deco_function] Saliendo... 


Podemos ver que la función `print_paya` tendrá el nuevo comportamiento cuando la llamemos.

In [26]:
print_paya()

[wrapper_function] Entrando... 
¡Aro, aro, aro!
[wrapper_function] Saliendo... 


Veamos un ejemplo, ahora, con una nueva función vegetariana.

In [27]:
@deco_function
def print_comida():
    print("Comí pimentones con huevo.")
    print("Comí empanadas vegetarianas.")

[deco_function] Entrando... 
[deco_function] Saliendo... 


Veamos el resultado.

In [28]:
print_comida()

[wrapper_function] Entrando... 
Comí pimentones con huevo.
Comí empanadas vegetarianas.
[wrapper_function] Saliendo... 


El decorador se aplicó satisfactoriamente.

Aplicar el decorador como:

In [30]:
@deco_function
def print_paya():
    print("¡Aro, aro, aro!")

[deco_function] Entrando... 
[deco_function] Saliendo... 


Es equivalente a haber ejecutado:


In [31]:
print_paya = deco_function(print_paya)

[deco_function] Entrando... 
[deco_function] Saliendo... 


### Decoradores para funciones con argumentos

Hemos visto cómo aplicar un decorador de dos formas equivalentes, pero los hemos aplicado sólo sobre funciones sin parámetros. Intentemos con una nueva función que, a diferencia de las anteriores, acepta un parámetro.

In [37]:
@deco_function
def print_bebida(bebida):
    print(f"Este dieciocho, me tomé dos litros de {bebida} al día.")

[deco_function] Entrando... 
[deco_function] Saliendo... 


Veamos nuevamente el resultado.

In [38]:
print_bebida("agua")

TypeError: wrapper_function() takes 0 positional arguments but 1 was given

Algo salió mal. El _wrapper_ no esperaba recibir un argumento. 

Podríamos resolver este problema colocando un parámetro a `wrapper_function` que está definido en nuestro decorador. Sin embargo, con esa solución sólo podremos decorar funciones que reciben exactamente un parámetro. Lo que a nosotros nos gustaría es poder decorar cualquier función, independiente de la cantidad de parámetros que reciba.

Para resolver este problema en forma definitiva, necesitamos utilizar `*args` y `**kwargs`.

- “¿Qué es `*args` y `**kwargs`?”  
    - Ellos son utilizados generalmente en la definición de funciones, y sirven para pasar una **cantidad variable** de argumentos.  

- “¿Y para qué me sirve eso?”  
    - Esto me será de gran utilidad en casos cuando yo no sepa, de antemano, cuántos argumentos me llegarán.



Volviendo al primer decorador, agreguémosles `*args` y `**kwargs`.

In [39]:
def deco_function(funcion_original):
    print("[deco_function] Entrando... ")
    
    ## Ahora la función 'envoltorio', es capaz de recibir una cantidad variable de argumentos
    def wrapper_function(*args, **kwargs):    
        print("[wrapper_function] Entrando... ")
        ## Esta cantidad variable de argumentos se le entrega a la función original
        funcion_original(*args, **kwargs)
        print("[wrapper_function] Saliendo... ")
    
    print("[deco_function] Saliendo... ")
    return wrapper_function


In [40]:
@deco_function
def print_bebida(bebida):
    print(f"Este dieciocho, me tomé dos litros de {bebida} por día.")

[deco_function] Entrando... 
[deco_function] Saliendo... 


Veamos nuevamente el resultado.

In [41]:
print_bebida("pipeño")

[wrapper_function] Entrando... 
Este dieciocho, me tomé dos litros de pipeño por día.
[wrapper_function] Saliendo... 


Ahora sí funcionó. 😀

### Decoradores con parámetros

Si queremos crear decoradores que acepten parámetros, debemos agregar un tercer nivel de funciones anidadas. Cada nivel tiene un rol:

- La función más externa es el constructor del decorador.
- La función intermedia es el decorador.
- La función más interna es la función modificada.

En general, esta estructura se ve de la siguiente manera:

In [42]:
# Función creadora de decoradores. 
# Recibe parámetros para personalizar nuestro decorador.
def my_decorator_constructor(dec_parameters):
    # Función decoradora. Recibe sólo una función.
    def my_decorator(function): 
        # Wrapper. Acá podemos leer los argumentos recibidos.
        # en las dos funciones que están en niveles superiores.
        def wrapped_function(*args, **kwargs):
            # Hacer algo aquí antes de aplicar la función.
            # Llamar a la función y obtener lo que retorna.
            res = function(*args, **kwargs)
            # Hacer algo después.
            # Retornar un valor.
            return res
        return wrapped_function # Retorna la sub función.
    return my_decorator # Retorna el decorador.

**Veamos un ejemplo.** Siguiendo con nuestros ejemplos dieciocheros, primero definamos un decorador etílico sin parámetros:

In [43]:
# Sleep pausará la ejecución del programa por la cantidad de segundos que se indique
from time import sleep as caña

def caña_de_pipeño(original_function):
    def wrapper(*args, **kwargs):
        caña(3)  # Pausa por 3 segundos
        return original_function(*args, **kwargs)

    return wrapper

Definamos una simple función decorada.

In [44]:
@caña_de_pipeño
def add_twelve(number):
    return number + 12

Veamos el resultado.

In [45]:
print(f"¡Feliz {add_twelve(6)} para todos!")

¡Feliz 18 para todos!


La función demoró en responder.

Ahora, imaginemos que buscamos implementar lo mismo, pero con un parámetro que indique qué bebida tomamos.

In [46]:
def caña_de(bebida):
    def deco_function(original_function):
        def wrapper(*args, **kwargs):
            if bebida == "vino":
                caña(2)
                print("Ayuda, por favor.")
            elif bebida == "pipeño":
                caña(3)
                print("¿Dónde estoy? ¿Quién soy?")
                args = (0, )  # Para acrecentar los efectos del pipeño,
                              # podemos también cambiar los parámetros.
            else:
                print("No hay caña.")
            return original_function(*args, **kwargs)
        
        return wrapper
    return deco_function

Definimos la misma función, pero ahora con el decorador recién definido.

In [47]:
@caña_de("pipeño")
def add_twelve(number):
    return number + 12

Analicemos los efectos del pipeño.

In [48]:
print(f"¡Feliz {add_twelve(6)} para todos!")

¿Dónde estoy? ¿Quién soy?
¡Feliz 12 para todos!


Claro… por lo que ocurrió el [12 de febrero de 1818](https://es.wikipedia.org/wiki/Acta_de_Independencia_de_Chile).

Podemos ver que además de demorar 3 segundos en dar la respuesta, los argumentos dados a `add_twelve` fueron reemplazados con una tupla con un cero. De hecho, si intentamos con otro valor:

In [49]:
print(f"¡Feliz {add_twelve(33)} para todos!")

¿Dónde estoy? ¿Quién soy?
¡Feliz 12 para todos!


Obtenemos el mismo resultado.